In [1]:
!python -c "import torch; print(torch.cuda.get_device_name(0));"
!python -c "import torch; print(torch.cuda.get_device_name(1));"
!python -c "import torch; print(torch.cuda.get_device_name(2));"
!python -c "import torch; print(torch.cuda.get_device_name(3));"
# !python -c "import torch; print(torch.cuda.get_device_name(4));"

Tesla V100-SXM2-32GB
Tesla V100-SXM2-32GB
Tesla V100-SXM2-32GB
Tesla V100-SXM2-32GB


In [1]:
import pandas as pd
from tqdm.auto import tqdm
tqdm.pandas()
tropes_def = pd.read_csv('https://raw.githubusercontent.com/armsp/trama/main/dataset/my_tropes.csv')
tropes_def

,TropeID,Trope,Description
0,t00079,AbusiveParents,Parents are supposed to be the protectors of c...
1,t00190,ActionGirl,An Action Girl is a female badass who is tough...
2,t00263,AdaptationDistillation,Some adaptations take a complex character or s...
3,t00265,AdaptationExpansion,This is the complete opposite of Compressed Ad...
4,t00270,AdaptationNameChange,Alice Andrews in the books becomes Alice Allen...
...,...,...,...
495,t30000,Redshirts,This is the Good Counterpart of Evil Minions a...
496,t30220,Bioaugmentation,The biological counterpart to Cyborging: inste...
497,t30286,XenoFiction,A few works of Science Fiction and Fantasy (an...
498,t30470,RecountedByTheMainCharacters,"Sometimes, ""this is a true story"" is part of t..."


In [2]:
def get_trope_by_id(tid):
    definition = tropes_def[tropes_def['TropeID'] == tid].iloc[0]['Description']
    name = tropes_def[tropes_def['TropeID'] == tid].iloc[0]['Trope']
    return (name, definition)
get_trope_by_id('t02187')

('BigBad',
 'The cause of all bad happenings in a story. A Big Bad could be a character with Evil Plans or it could be an omnipresent situation, such as a comet heading towards the Earth. In a serial story, the Big Bad exerts an effect across a number of episodes, even an entire season.\nThis trope is not a catch-all term for the biggest, ugliest villain of any given story. In fact, it doesn\'t have to be a villain at all, as we just said. If it is a villain, though, it should be identified correctly; the badass leader of the outlaw gang that causes the most personal trouble is not the Big Bad. The railroad tycoon who is using the gang as muscle is the Big Bad. The Man Behind the Man is very common for this trope, leaving the reveal of the big bad as The Chessmaster behind it all and proving themselves far more clever and resourceful than the Villain of the Week. Sometimes the Big Bad is the grand enemy of an entire franchise as an Overarching Villain. At other times, the Big Bad is an

In [3]:
stories_df = pd.read_csv('./stories_with_summary.csv')
stories_df

,Title,Author,story,summary_gpt_3_5,summary_gpt_3_5_t_14,sid
0,The Bear Came Over the Mountain,Alice Munro,"Fiona lived in her parents’ house, in the town...",Fiona and Grant meet in university and quickly...,Fiona is living in a big disorderly house with...,s0
1,The Boss,Robert Coover,"The gunman lights a cigarette, watches despond...","In this story, the gunman finds himself alone ...","In this story, the protagonist, referred to as...",s1
2,The Hairless Are Careless,Colin Barrett,"“Prejudicial of me, I know,” said Tommy, noddi...","Tommy, Ernie, and Luke are discussing the bald...","In this story, three men, Tommy, Ernie, and Lu...",s2
3,Smithereens,Aleksandar Hemon,"Near our mountain cabin, in Jahorina, there wa...",The story is set near a mountain cabin in Jaho...,The story is about the narrator's childhood me...,s3
4,Where Are You?,Joyce Carol Oates,The husband had got into the habit of calling ...,"In this story, the husband has developed a hab...","In this short story, the writer depicts a dete...",s4
...,...,...,...,...,...,...
80,Mrs. Packletide's Tiger,H. H. Munro,It was Mrs. Packletide's pleasure and intentio...,"Mrs. Packletide, driven by her dislike for Loo...",Mrs. Packletide decides that she wants to shoo...,s80
81,One of These Days,Gabriel Garcia Marquez,Monday dawned warm and rainless. Aurelio Escov...,Aurelio Escovar is a dentist who operates with...,The story is set on a warm and rainless Monday...,s81
82,A RETRIEVED REFORMATION,O. Henry,"A guard came to the prison shoe-shop, where Ji...","In this story, a man named Jimmy Valentine is ...","Jimmy Valentine, a skilled safe-cracker, is un...",s82
83,War of the Clowns,Mia Couto,One time two clowns set themselves to arguing....,"In this story, two clowns engage in an argumen...","In the story, two clowns engage in a playful a...",s83


In [4]:
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer, pipeline
torch.cuda.empty_cache() 
# from transformers import AutoTokenizer, AutoModelForCausalLM

hf_model = "/cluster/work/lawecon/Work/raj/llama2models/70b-chat-hf" 

tokenizer = LlamaTokenizer.from_pretrained(hf_model)

model = LlamaForCausalLM.from_pretrained(
    hf_model,
    device_map="auto",
    torch_dtype=torch.float16,
    #  use_auth_token=True,
    load_in_8bit=True,
    # load_in_4bit=False,
    rope_scaling={"type": "dynamic", "factor": 2.0},
    # max_memory={0:"40.0GiB", 1:"40.0GiB", 2:"40.0GiB", 3:"40.0GiB"},
    # max_memory={0:"32.0GiB"},
    # max_memory={0:"40.0GiB"},
    # max_memory={0:"32.0GiB", 1:"32.0GiB"},
    max_memory={0:"32.0GiB", 1:"32.0GiB", 2:"32.0GiB", 3:"32.0GiB", 4:"32.0GiB", 5:"32.0GiB"},
    do_sample=True, #necessary to make temperature work
    # temperature=0.7 # ?? failed
)

2023-11-11 17:50:15.922607: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-11 17:50:15.922653: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-11 17:50:15.922691: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-11 17:50:15.934183: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-11 17:50:29.626210: W tensorflow/compiler/

Loading checkpoint shards:   0%|          | 0/15 [00:00<?, ?it/s]

In [ ]:
# Use a pipeline for later
pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.float16,
                device_map="auto",
                do_sample=True,
                top_k=5,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id,
                max_length=4096,
                return_full_text=False,
                # temperature=0.4 # ??
                )
# print(pipeline)
# print("######")
# @print_function_name
def generate(prompt, temp=0.6):
    with torch.autocast("cuda", dtype=torch.float16):
        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
        outputs = model.generate(
            **inputs,
            max_new_tokens=4096,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id,
            temperature = temp #??
        )
        # final_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        final_outputs = tokenizer.batch_decode(outputs[:, inputs.shape[1]:])[0]
        #       generated_text = final_outputs[len(prompt):]
        # final_outputs = cut_off_text(final_outputs, '</s>')
        print(final_outputs)
        # torch.cuda.empty_cache() 
        print("#####################################################")
        return final_outputs

# @print_function_name
def pipe_generate(prompt, temp=0.6):
    sequences = pipe(prompt, temperature=temp)
    gen_text = []
    for seq in sequences:
        gen_text.append(seq['generated_text'])
        # print(seq['generated_text'])
    torch.cuda.empty_cache() 
    print("#####################################################")
    return gen_text

In [ ]:
generate('Tell me about the OurVampiresAreDifferent trope')

In [19]:
out = pipe_generate('Tell me about the OurVampiresAreDifferent trope', temp=0.5)
out[0]

#####################################################


'. What are some examples of vampires in fiction that subvert or play with common vampire tropes?\n\nOurVampiresAreDifferent is a popular trope in fiction that involves reimagining or subverting common vampire tropes. Here are some examples of vampires in fiction that play with or subvert common vampire tropes:\n\n1. The Vampire Lestat from Anne Rice\'s "Vampire Chronicles" series: Lestat is a charismatic, flamboyant, and rebellious vampire who defies many traditional vampire tropes. He walks in sunlight, doesn\'t have fangs, and doesn\'t drink human blood. Instead, he feeds on rats and other small animals.\n2. The vampires in "The Strain" by Guillermo del Toro and Chuck Hogan: These vampires are more like zombies than traditional vampires. They are created by a virus that turns people into mindless, bloodthirsty creatures with no supernatural powers.\n3. The vampires in "True Blood" by Charlaine Harris: These vampires have "come out of the coffin" and are living openly among humans. T

In [6]:
global prompt, system_prompt, user_msg
prompt = """<s>[INST] <<SYS>>
{}
<</SYS>>

{} [/INST]"""

system_prompt = """Analyze if the given trope exists in the provided short story. The expected output has to be - 
{
    "exists": {yes or no},
    "reason": {explanation}
}
"""
user_msg = """Trope: {}\n
Story:\n
{}
"""

def check_trope_exists(row):
    a,b = row
    print(b)
    story = a
    tropes = b
    tropes_validated = []
    for trope in tropes:
        # trope_def = get_trope_by_id(trope)
        # trope_def goes in system_prompt
        # system_prompt.format()
        # story goes in user msg
        u_m = user_msg.format(trope, story)
        # form the prompt
        p = prompt.format(system_prompt, u_m)
        # query the model
        generate(p, temp=0.6)
        # parse the output and save it to tropes_validated
    return tropes_validated
        
# prompt.format(system_p.strip(), user_m.format(story.strip()))

In [7]:
s = stories_df[stories_df['sid']=="s15"].iloc[0].story
t = ['HotSkittyOnWailordAction', 'BigBad', 'ProudWarriorRaceGuy', 'OurVampiresAreDifferent', 'ShoutOut', 'FairplayWhodunnit', 'Foreshadowing', 'DramaticIrony', 'GenderBlenderName', 'FantasyCounterpartCulture']

In [8]:
out = check_trope_exists((s,t))
print(out)

['HotSkittyOnWailordAction', 'BigBad', 'ProudWarriorRaceGuy', 'OurVampiresAreDifferent', 'ShoutOut', 'FairplayWhodunnit', 'Foreshadowing', 'DramaticIrony', 'GenderBlenderName', 'FantasyCounterpartCulture']
[INST] <<SYS>>
Analyze if the given trope exists in the provided short story. The expected output has to be - 
{
    "exists": {yes or no},
    "reason": {explanation}
}

<</SYS>>

Trope: HotSkittyOnWailordAction

Story:

Just behind the kingdom that failed ran a nice little river. It was a clear, lovely stream, and many fish lived in it. Weeds grew there, too, and the fish ate the weeds. The fish didn’t care whether or not the kingdom had failed, of course. Whether it was a kingdom or a republic made no difference to them. They didn’t vote or pay taxes. Makes no difference to us, they figured.

I washed my feet in the stream. A short soak in the icy-cold water turned them red. From the stream, you could see the walls and the tower of the castle in the kingdom that failed. The two-co

In [71]:
import ast
import distutils
global prompt, system_prompt, user_msg
def strtobool (val):
    """Convert a string representation of truth to true (1) or false (0).
    True values are 'y', 'yes', 't', 'true', 'on', and '1'; false values
    are 'n', 'no', 'f', 'false', 'off', and '0'.  Raises ValueError if
    'val' is anything else.
    """
    val = val.lower()
    if val in ('y', 'yes', 't', 'true', 'on', '1', '"yes"', "'yes'"):
        return 1
    elif val in ('n', 'no', 'f', 'false', 'off', '0', '"no"', "'no'"):
        return 0
    else:
        raise ValueError("invalid truth value %r" % (val,))
prompt = """<s>[INST] <<SYS>>
{}
<</SYS>>

{} [/INST]"""

system_prompt = """You always communicate by making a valid python JSON string. The expected output from you should follow the structure:\n 
    {
        "exists": {yes or no},
        "reason": {evidence of the trope in the context of the provided story}
    }
"""
user_msg = """Analyze if the given trope exists or not in the story. \nTrope: {}\nStory:\n{}"""

def check_trope_exists(row):
    # print(row[1])
    a = row['story']
    b = row['tropes']
    # print(b)
    story = a
    tropes = b
    tropes_validated = []
    for trope in tropes:
        trope_name, trope_def = get_trope_by_id(trope)
        # trope_def goes in system_prompt
        # system_prompt.format()
        # story goes in user msg
        u_m = user_msg.format(trope_name, story)
        # form the prompt
        p = prompt.format(system_prompt, u_m)
        # query the model
        # generate(p, temp=0.7)
        out = pipe_generate(p, temp=0.6)
        out = out[0]
        # out = generate(p, temp=0.6)
        print(out)
        # p_out = ast.literal_eval(out)
        # p_out = {}
        # exists = out.split(',')[0].split(': ')[1].strip()
        # p_out['exists'] = strtobool(exists)
        # p_out['reason'] = out.split(',',1)[1].split(':',1)[1].split('}')[0].strip()
        # print(p_out)
        
        p_out = {}
        target = out.split('{')[1].split('}')[0]
        exists = target.split(',')[0].split(': ')[1].strip()
        p_out['exists'] = strtobool(exists)
        p_out['reason'] = target.split(',',1)[1].split(':',1)[1].split('}')[0].strip()
        print(p_out)
        tropes_validated.append(p_out)
        # parse the output and save it to tropes_validated
        
    return tropes_validated

# prompt.format(system_p.strip(), user_m.format(story.strip()))

In [43]:
s = stories_df[stories_df['sid']=="s15"].iloc[0].story
t = ['HotSkittyOnWailordAction', 'BigBad', 'ProudWarriorRaceGuy', 'OurVampiresAreDifferent', 'ShoutOut', 'FairplayWhodunnit', 'Foreshadowing', 'DramaticIrony', 'GenderBlenderName', 'FantasyCounterpartCulture']
check_trope_exists((s,t))

<s>[INST] <<SYS>>
You always communicate by making a valid python JSON string. The expected output from you should follow the structure:
 
    {
        "exists": {yes or no},
        "reason": {evidence of the trope in the context of the provided story}
    }

<</SYS>>

Analyze if the given trope exists or not in the story. 
Trope: HotSkittyOnWailordAction
Story:
Just behind the kingdom that failed ran a nice little river. It was a clear, lovely stream, and many fish lived in it. Weeds grew there, too, and the fish ate the weeds. The fish didn’t care whether or not the kingdom had failed, of course. Whether it was a kingdom or a republic made no difference to them. They didn’t vote or pay taxes. Makes no difference to us, they figured.

I washed my feet in the stream. A short soak in the icy-cold water turned them red. From the stream, you could see the walls and the tower of the castle in the kingdom that failed. The two-colored flag still flew from the tower, fluttering in the breez

TypeError: 'NoneType' object is not subscriptable

In [6]:
from ast import literal_eval
story_w_trope = pd.read_csv('story_with_tropes_story.csv', converters={"tropes": literal_eval})
story_w_trope

,Title,Author,story,summary_gpt_3_5,summary_gpt_3_5_t1,sid,tropes
0,The Bear Came Over the Mountain,Alice Munro,"Fiona lived in her parents’ house, in the town...",Fiona and Grant are a married couple. Fiona's ...,Fiona and Grant are a married couple who live ...,s0,[]
1,The Boss,Robert Coover,"The gunman lights a cigarette, watches despond...",The story follows a gunman who is waiting in a...,The story follows a gunman who finds himself a...,s1,"[t08393, t02187, t18297, t03644, t26668, t2022..."
2,The Hairless Are Careless,Colin Barrett,"“Prejudicial of me, I know,” said Tommy, noddi...","Tommy, Ernie, and Luke are discussing bald peo...","Tommy and his friends, Ernie and Luke, are at ...",s2,"[t20214, t02282, t14940, t02187, t24108, t0839..."
3,Smithereens,Aleksandar Hemon,"Near our mountain cabin, in Jahorina, there wa...",The story is set near a mountain cabin in Jaho...,The narrator recalls a summer from their child...,s3,"[t08393, t26668, t03644, t02187, t15274, t0659..."
4,Where Are You?,Joyce Carol Oates,The husband had got into the habit of calling ...,"In this story, a husband and wife have develop...","In this story, a husband and wife have a pecul...",s4,[]
...,...,...,...,...,...,...,...
79,The Mouse,H. H. Munro,"Theodoric Voler had been brought up, from infa...","Theodoric Voler, a man who has been sheltered ...","Theodoric Voler, a man who had been sheltered ...",s79,"[t02187, t06593, t20214, t00265, t08393, t1059..."
80,Mrs. Packletide's Tiger,H. H. Munro,It was Mrs. Packletide's pleasure and intentio...,"Mrs. Packletide, motivated by jealousy of Loon...",Mrs. Packletide is determined to shoot a tiger...,s80,"[t02187, t08393, t00265, t06175, t20214, t1494..."
81,One of These Days,Gabriel Garcia Marquez,Monday dawned warm and rainless. Aurelio Escov...,"Aurelio Escovar, a dentist without a degree, o...","Aurelio Escovar, a dentist with no formal educ...",s81,"[t18297, t08393, t26723, t03644, t26668, t0659..."
82,A RETRIEVED REFORMATION,O. Henry,"A guard came to the prison shoe-shop, where Ji...","Jimmy Valentine, a skilled safecracker, is rel...","Jimmy Valentine, a skilled safecracker, is rel...",s82,"[t06452, t08905, t08393, t20214, t06175, t1644..."


In [47]:
story_w_trope['validation_70b'] = story_w_trope.progress_apply(check_trope_exists, axis=1)
story_w_trope

  0%|          | 0/84 [00:00<?, ?it/s]

#####################################################
  {
"exists": yes,
"reason": The story contains several elements that foreshadow future events. For example, the gunman's watch with a gold coin face and hair-thin hands is a symbol of time running out, which foreshadows the impending doom that awaits him and the woman he has just met. The street lamp that comes on and off, casting shadows that seem to have a life of their own, creates an ominous atmosphere that suggests something bad is about to happen. The gunman's specialty of knocking off targets in public places and his concern about getting caught, which is hinted at through his unease when the woman passes by, foreshadow the danger lurking in the shadows. The woman's hesitation and nervous glances also hint at a sense of impending doom. The gunman's line, "It won't do any good, lady. But I like it," suggests that he has a sense of inevitability about what is about to happen, which is confirmed when the woman is shot and kille

/cluster/home/sharaj/.local/lib/python3.9/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


#####################################################
  {
"exists": true,
"reason": "The story contains a reference to the trope of the 'bald man' being associated with bad luck or negative consequences. The character Tommy attributes the baldness of the man they see at the petrol station to'some inherent deficiency of character' and later connects it to the fact that the man is an American, implying that he is not to be trusted. This is an example of the ShoutOut trope as it is a nod to the stereotype that bald men are suspicious or untrustworthy."
}
{'exists': 1, 'reason': '"The story contains a reference to the trope of the \'bald man\' being associated with bad luck or negative consequences. The character Tommy attributes the baldness of the man they see at the petrol station to\'some inherent deficiency of character\' and later connects it to the fact that the man is an American, implying that he is not to be trusted. This is an example of the ShoutOut trope as it is a nod to the 

KeyboardInterrupt: 

In [53]:
prompt = """<s>[INST] <<SYS>>
{}
<</SYS>>

{} [/INST]"""

system_prompt = """You always communicate using a valid python JSON string. You will be given a trope and a story. Analyze if the given trope exists or not in the story. The expected output from you should follow the structure:\n 
    {
        "trope_exists": {'yes' if the trope exists, 'no' if the trope does not exist},
        "evidence": {explain concisely why in the context of the story}
    }
"""
user_msg = """\nTrope: {}\nStory:\n{}"""

def check_trope_exists(row):
    print(row)
    # print(b)
    story = row[0]
    tropes = row[1]
    tropes_validated = []
    for trope in tropes:
        # trope_name, trope_def = get_trope_by_id(trope)
        # trope_def goes in system_prompt
        # system_prompt.format()
        # story goes in user msg
        u_m = user_msg.format(trope.strip(), story.strip())
        # form the prompt
        p = prompt.format(system_prompt, u_m)
        # query the model
        # generate(p, temp=0.7)
        out = pipe_generate(p, temp=0.6)
        out = out[0]
        # out = generate(p, temp=0.6)
        print(out)
        # p_out = ast.literal_eval(out)
        # p_out = {}
        # exists = out.split(',')[0].split(': ')[1].strip()
        # p_out['exists'] = strtobool(exists)
        # p_out['reason'] = out.split(',',1)[1].split(':',1)[1].split('}')[0].strip()
        # print(p_out)
        
        p_out = {}
        target = out.split('{')[1].split('}')[0]
        exists = target.split(',')[0].split(': ')[1].strip()
        # p_out['tid'] = trope
        p_out['tname'] = trope
        p_out['exists'] = strtobool(exists)
        p_out['reason'] = target.split(',',1)[1].split(':',1)[1].split('}')[0].strip()
        print(p_out)
        tropes_validated.append(p_out)
        # parse the output and save it to tropes_validated
        
    return tropes_validated

# prompt.format(system_p.strip(), user_m.format(story.strip()))
s = stories_df[stories_df['sid']=="s15"].iloc[0].story
# print(s)
t = ['HotSkittyOnWailordAction', 'BigBad', 'ProudWarriorRaceGuy', 'OurVampiresAreDifferent', 'ShoutOut', 'FairplayWhodunnit', 'Foreshadowing', 'DramaticIrony', 'GenderBlenderName', 'FantasyCounterpartCulture']
check_trope_exists((s,t))

('Just behind the kingdom that failed ran a nice little river. It was a clear, lovely stream, and many fish lived in it. Weeds grew there, too, and the fish ate the weeds. The fish didn’t care whether or not the kingdom had failed, of course. Whether it was a kingdom or a republic made no difference to them. They didn’t vote or pay taxes. Makes no difference to us, they figured.\n\nI washed my feet in the stream. A short soak in the icy-cold water turned them red. From the stream, you could see the walls and the tower of the castle in the kingdom that failed. The two-colored flag still flew from the tower, fluttering in the breeze. Everyone passing along the riverbanks would see the flag and say, “Hey, look at that. It’s the flag of the kingdom that failed.”\n\nQ and I are friends—or should I say, were friends in college. It’s been more than ten years since the two of us did anything that friends do. Which is why I use the past tense. Anyhow, we were friends.\n\nWhenever I try to tell 

[{'tname': 'HotSkittyOnWailordAction',
  'exists': 0,
  'reason': '"The story does not contain any elements of the HotSkittyOnWailordAction trope. The story is about a man named Q who is described as handsome, popular, and talented, but there is no mention of him engaging in any sort of romantic or sexual activity with a woman, let alone a woman who is described as a "HotSkitty." The story does not contain any language or imagery that would suggest a sexual or romantic connection between Q and the woman he is talking to. Therefore, the trope does not exist in this story."'},
 {'tname': 'BigBad',
  'exists': 0,
  'reason': '"The story does not contain any elements of the BigBad trope. Q, the main character, is portrayed as a handsome, charming, and successful man, but he does not exhibit any behavior that would suggest he is a villain or an antagonist. In fact, he is shown to be a caring and empathetic person, trying to explain the situation to the woman in a sincere way. The story does

In [56]:
prompt = """<s>[INST] <<SYS>>
{}
<</SYS>>

{} [/INST]"""

system_prompt = """You always communicate using a valid python JSON string. You will be given a trope and a story. Analyze if the given trope exists or not in the story. The expected output from you should follow the structure:\n 
    {
        "trope_exists": {'yes' if the trope exists, 'no' if the trope does not exist},
        "evidence": {explain concisely why in the context of the story}
    }
"""
user_msg = """\nTrope: {}\nStory:\n{}"""
def check_trope_exists(row):
    print(row)
    story = row[0]
    tropes = row[1]
    tropes_validated = []
    for trope in tropes:
        trope_name, trope_def = get_trope_by_id(trope)
        u_m = user_msg.format(trope_name.strip(), story.strip())
        # form the prompt
        p = prompt.format(system_prompt, u_m)
        # query the model
        # generate(p, temp=0.7)
        out = pipe_generate(p, temp=0.6)
        out = out[0]
        print(out)
        p_out = {}
        target = out.split('{')[1].split('}')[0]
        exists = target.split(',')[0].split(': ')[1].strip()
        p_out['tname'] = trope_name
        p_out['exists'] = strtobool(exists)
        p_out['reason'] = target.split(',',1)[1].split(':',1)[1].split('}')[0].strip()
        print(p_out)
        tropes_validated.append(p_out)
        # parse the output and save it to tropes_validated
    return tropes_validated
s = stories_df[stories_df['sid']=="s1"].iloc[0].story
# print(s)
t = story_w_trope[story_w_trope['sid']=="s1"].iloc[0].tropes
check_trope_exists((s,t))

('The gunman lights a cigarette, watches despondently as dusk falls upon the empty alley. He is alone in a lonely place, summoned here to receive instructions from a master criminal known only as the Boss, but the Boss isn’t here. No one is. It’s spooky. He feels like a marked man. The Boss is known for his ruthlessness. When he orders a killing, someone dies. The gunman would like there to be witnesses for what happens next, but the alley’s deserted.\n\nHe glances at his watch, a gift from the Boss. Face a gold coin, no numbers. A joke, probably: time is money. Or, maybe, money is time; it depends on what you’re short of. The Boss is a great joker. The watch hands are hair-thin, like the edge of a razor blade, hard to see, especially in this fading light. There and not there, like time itself. Which is perhaps not being clocked—perhaps that’s what the numberless face is saying. How can you measure the shit you’re buried in? He doesn’t know what keeps the watch running. Battery inside,

[{'tname': 'Foreshadowing',
  'exists': 1,
  'reason': '"The story contains several elements of foreshadowing. For example, the gunman\'s watch with a gold coin face and hair-thin hands is described as \'like time itself\' and \'perhaps that’s what the numberless face is saying. How can you measure the shit you’re buried in?\' This could foreshadow the idea that time is running out for the gunman. Additionally, the woman\'s sudden appearance and the gunman\'s immediate suspicion of her suggest that she may be a witness or a victim, foreshadowing her eventual death. The street lamp overhead coming on and the shadows it casts also create a sense of foreboding, foreshadowing the violence that will soon occur."'},
 {'tname': 'BigBad',
  'exists': 1,
  'reason': '"The gunman\'s interaction with the woman in the alley, who sacrifices herself to save him from an assassin, exhibits the characteristics of the BigBad trope. The gunman, who is a hitman for a criminal organization, is the protagon

In [57]:
prompt = """<s>[INST] <<SYS>>
{}
<</SYS>>

{} [/INST]"""

system_prompt = """You always communicate using a valid python JSON string. You will be given a trope and a story. Analyze carefully if the given trope exists or not in the story. The expected output from you should follow the structure:\n 
    {
        "trope_exists": {'yes' if the trope exists, 'no' if the trope does not exist},
        "evidence": {reason why the trope exists or not in the context of the story}
    }
"""
user_msg = """\nTrope: {}\nStory:\n{}"""
def check_trope_exists(row):
    print(row)
    story = row[0]
    tropes = row[1]
    tropes_validated = []
    for trope in tropes:
        trope_name, trope_def = get_trope_by_id(trope)
        u_m = user_msg.format(trope_name.strip(), story.strip())
        # form the prompt
        p = prompt.format(system_prompt, u_m)
        # query the model
        # generate(p, temp=0.7)
        out = pipe_generate(p, temp=0.6)
        out = out[0]
        print(out)
        p_out = {}
        target = out.split('{')[1].split('}')[0]
        exists = target.split(',')[0].split(': ')[1].strip()
        p_out['tname'] = trope_name
        p_out['exists'] = strtobool(exists)
        p_out['reason'] = target.split(',',1)[1].split(':',1)[1].split('}')[0].strip()
        print(p_out)
        tropes_validated.append(p_out)
        # parse the output and save it to tropes_validated
    return tropes_validated
s = stories_df[stories_df['sid']=="s1"].iloc[0].story
# print(s)
t = story_w_trope[story_w_trope['sid']=="s1"].iloc[0].tropes
check_trope_exists((s,t))

('The gunman lights a cigarette, watches despondently as dusk falls upon the empty alley. He is alone in a lonely place, summoned here to receive instructions from a master criminal known only as the Boss, but the Boss isn’t here. No one is. It’s spooky. He feels like a marked man. The Boss is known for his ruthlessness. When he orders a killing, someone dies. The gunman would like there to be witnesses for what happens next, but the alley’s deserted.\n\nHe glances at his watch, a gift from the Boss. Face a gold coin, no numbers. A joke, probably: time is money. Or, maybe, money is time; it depends on what you’re short of. The Boss is a great joker. The watch hands are hair-thin, like the edge of a razor blade, hard to see, especially in this fading light. There and not there, like time itself. Which is perhaps not being clocked—perhaps that’s what the numberless face is saying. How can you measure the shit you’re buried in? He doesn’t know what keeps the watch running. Battery inside,

[{'tname': 'Foreshadowing',
  'exists': 1,
  'reason': '"The story contains foreshadowing elements, such as the gunman\'s comment about the Boss\'s plans haven\'t happened yet and the ominous atmosphere created by the deserted alley and the swaying street lamp. The gunman\'s unease and the woman\'s warning also contribute to the sense of foreboding. The story also ends with a dramatic and unexpected event, the woman\'s death, which adds to the sense of tension and danger that has been building throughout the story."'},
 {'tname': 'BigBad',
  'exists': 1,
  'reason': '"The gunman\'s conversation with the woman in the alley, the way he looks at her and the way she looks at him, and the way he feels caught up in something fundamental and erotic, all suggest that the trope of BigBad is present in the story. The gunman\'s actions and feelings towards the woman show that he is not just a heartless criminal, but also a complex character with his own motivations and emotions, making him a BigB

In [58]:
prompt = """<s>[INST] <<SYS>>
{}
<</SYS>>

{} [/INST]"""

system_prompt = """You always communicate using a valid python JSON string. You will be given a trope and a story. Analyze carefully if the given trope exists or not in the story. The expected output from you should follow the structure:\n 
    {
        "trope_exists": {'yes' if the trope exists, 'no' if the trope does not exist},
        "evidence": {reason why the trope exists or does not exist in the context of the story}
    }
"""
user_msg = """\nTrope: {}\nStory:\n{}"""
def check_trope_exists(row):
    print(row)
    story = row[0]
    tropes = row[1]
    tropes_validated = []
    for trope in tropes:
        trope_name, trope_def = get_trope_by_id(trope)
        u_m = user_msg.format(trope_name.strip(), story.strip())
        # form the prompt
        p = prompt.format(system_prompt, u_m)
        # query the model
        # generate(p, temp=0.7)
        out = pipe_generate(p, temp=0.7)
        out = out[0]
        print(out)
        p_out = {}
        target = out.split('{')[1].split('}')[0]
        exists = target.split(',')[0].split(': ')[1].strip()
        p_out['tname'] = trope_name
        p_out['exists'] = strtobool(exists)
        p_out['reason'] = target.split(',',1)[1].split(':',1)[1].split('}')[0].strip()
        print(p_out)
        tropes_validated.append(p_out)
        # parse the output and save it to tropes_validated
    return tropes_validated
s = stories_df[stories_df['sid']=="s1"].iloc[0].story
# print(s)
t = story_w_trope[story_w_trope['sid']=="s1"].iloc[0].tropes
check_trope_exists((s,t))

('The gunman lights a cigarette, watches despondently as dusk falls upon the empty alley. He is alone in a lonely place, summoned here to receive instructions from a master criminal known only as the Boss, but the Boss isn’t here. No one is. It’s spooky. He feels like a marked man. The Boss is known for his ruthlessness. When he orders a killing, someone dies. The gunman would like there to be witnesses for what happens next, but the alley’s deserted.\n\nHe glances at his watch, a gift from the Boss. Face a gold coin, no numbers. A joke, probably: time is money. Or, maybe, money is time; it depends on what you’re short of. The Boss is a great joker. The watch hands are hair-thin, like the edge of a razor blade, hard to see, especially in this fading light. There and not there, like time itself. Which is perhaps not being clocked—perhaps that’s what the numberless face is saying. How can you measure the shit you’re buried in? He doesn’t know what keeps the watch running. Battery inside,

[{'tname': 'Foreshadowing',
  'exists': 1,
  'reason': '"The story contains foreshadowing elements, such as the gunman\'s unease and the ominous atmosphere, which hint at the tragic events that will unfold. The author also employs symbolism, such as the gold watch with the blank face, which suggests that time is running out for the characters. Additionally, the woman\'s sudden appearance and her eerie behavior, such as her slow walk towards the darkness and her eventual betrayal, further heighten the tension and foreboding in the scene."'},
 {'tname': 'BigBad',
  'exists': 1,
  'reason': '"The gunman\'s encounter with the woman in the alley and the events that unfold demonstrate the trope of \'BigBad\' as the gunman is the one who is supposed to be in control and taking charge, but instead, he is left confused and powerless as the woman takes the lead and ultimately meets her demise. Additionally, the gunman\'s confusion and rage at the end of the story suggest that he has been outsmar

In [60]:
import re
name = "CamelCaseTest123"
splitted = re.sub('([A-Z][a-z]+)', r' \1', re.sub('([A-Z]+)', r' \1', name)).split()
" ".join(splitted)

'Camel Case Test123'

In [65]:
prompt = """<s>[INST] <<SYS>>
{}
<</SYS>>

{} [/INST]"""

system_prompt = """You always communicate using a valid python JSON string. You will be given a trope and a story. Analyze carefully if the given trope exists or not in the story. The expected output from you should follow the structure:\n 
    {
        "trope_exists": {'yes' if the trope exists, 'no' if the trope does not exist},
        "evidence": {reason why the trope exists or does not exist in the context of the story}
    }
"""
user_msg = """\nTrope: {}\nStory:\n{}"""
def check_trope_exists(row):
    print(row)
    story = row[0]
    tropes = row[1]
    tropes_validated = []
    for trope in tropes:
        trope_name, trope_def = get_trope_by_id(trope)
        splitted = re.sub('([A-Z][a-z]+)', r' \1', re.sub('([A-Z]+)', r' \1', trope_name)).split()
        trope_name = " ".join(splitted)
        u_m = user_msg.format(trope_name.strip(), story.strip())
        # form the prompt
        p = prompt.format(system_prompt, u_m)
        # query the model
        # generate(p, temp=0.7)
        out = pipe_generate(p, temp=0.65)
        out = out[0]
        print(out)
        p_out = {}
        target = out.split('{')[1].split('}')[0]
        exists = target.split(',')[0].split(': ')[1].strip()
        p_out['tname'] = trope_name
        p_out['exists'] = strtobool(exists)
        p_out['reason'] = target.split(',',1)[1].split(':',1)[1].split('}')[0].strip()
        print(p_out)
        tropes_validated.append(p_out)
        # parse the output and save it to tropes_validated
    return tropes_validated
s = stories_df[stories_df['sid']=="s1"].iloc[0].story
# print(s)
t = story_w_trope[story_w_trope['sid']=="s1"].iloc[0].tropes
check_trope_exists((s,t))

('The gunman lights a cigarette, watches despondently as dusk falls upon the empty alley. He is alone in a lonely place, summoned here to receive instructions from a master criminal known only as the Boss, but the Boss isn’t here. No one is. It’s spooky. He feels like a marked man. The Boss is known for his ruthlessness. When he orders a killing, someone dies. The gunman would like there to be witnesses for what happens next, but the alley’s deserted.\n\nHe glances at his watch, a gift from the Boss. Face a gold coin, no numbers. A joke, probably: time is money. Or, maybe, money is time; it depends on what you’re short of. The Boss is a great joker. The watch hands are hair-thin, like the edge of a razor blade, hard to see, especially in this fading light. There and not there, like time itself. Which is perhaps not being clocked—perhaps that’s what the numberless face is saying. How can you measure the shit you’re buried in? He doesn’t know what keeps the watch running. Battery inside,

[{'tname': 'Foreshadowing',
  'exists': 1,
  'reason': '"The story contains foreshadowing elements, such as the gunman\'s mention of the Boss\'s ruthlessness and the ominous atmosphere created by the deserted alley and the fading light. The gunman\'s internal monologue also hints at a larger plan or threat, as he mentions the Boss\'s plans that haven\'t happened yet. Additionally, the woman\'s sudden appearance and her eventual death by a gunshot wound suggest that she was meant to be a witness or a victim, further foreshadowing the danger that the gunman is in."'},
 {'tname': 'Big Bad',
  'exists': 1,
  'reason': '"The story features a Big Bad in the form of the master criminal known only as the Boss. The Boss is described as ruthless and powerful, able to summon the gunman to carry out his orders. The gunman is fearful of the Boss and his power, indicating that the Boss is a formidable villain. The story also implies that the Boss has a network of operatives and is able to manipulate

In [27]:
# torch.cuda.empty_cache()
prompt = """<s>[INST] <<SYS>>
{}
<</SYS>>

{} [/INST]"""

system_prompt = """You always communicate using a valid python JSON string. You will be given a trope and a story. Analyze carefully if the given trope exists or not in the story. The expected output from you should follow the structure:\n 
    {
        "trope_exists": {'yes' if the trope exists, 'no' if the trope does not exist},
        "evidence": {reason why the trope exists or does not exist in the context of the story}
    }
"""
user_msg = """Trope: {}\nStory:\n{}"""
def check_trope_exists(row):
    print(row)
    story = row[0]
    tropes = row[1]
    tropes_validated = []
    for trope in tropes:
        trope_name, trope_def = get_trope_by_id(trope)
        splitted = re.sub('([A-Z][a-z]+)', r' \1', re.sub('([A-Z]+)', r' \1', trope_name)).split()
        trope_name = " ".join(splitted)
        u_m = user_msg.format(trope_name.strip(), story.strip())
        # form the prompt
        p = prompt.format(system_prompt, u_m)
        # query the model
        # generate(p, temp=0.7)
        # print(p)
        out = pipe_generate(p, temp=0.45)# 0.50, .55 does a good job but it still hallucinates, 0.45 is great, 0.35 is also good #stick to 0.45
        out = out[0]
        # out = generate(p, temp=0.45)
        print(trope_name, out)
        p_out = {}
        # target = out.split('{')[1].split('}')[0]
        # exists = target.split(',')[0].split(': ')[1].strip()
        # p_out['tname'] = trope_name
        # p_out['exists'] = strtobool(exists)
        # p_out['reason'] = target.split(',',1)[1].split(':',1)[1].split('}')[0].strip()
        # print(p_out)
        tropes_validated.append(p_out)
        # parse the output and save it to tropes_validated
    return tropes_validated
s = story_w_trope[story_w_trope['sid']=="s82"].iloc[0].story
# print(s)
t = story_w_trope[story_w_trope['sid']=="s82"].iloc[0].tropes
check_trope_exists((s,t))

('A guard came to the prison shoe-shop, where Jimmy Valentine was assiduously stitching uppers, and escorted him to the front office. There the warden handed Jimmy his pardon, which had been signed that morning by the governor. Jimmy took it in a tired kind of way. He had served nearly ten months of a four year sentence. He had expected to stay only about three months, at the longest. When a man with as many friends on the outside as Jimmy Valentine had is received in the "stir" it is hardly worth while to cut his hair.\n\n"Now, Valentine," said the warden, "you\'ll go out in the morning. Brace up, and make a man of yourself. You\'re not a bad fellow at heart. Stop cracking safes, and live straight."\n\n"Me?" said Jimmy, in surprise. "Why, I never cracked a safe in my life."\n\n"Oh, no," laughed the warden. "Of course not. Let\'s see, now. How was it you happened to get sent up on that Springfield job? Was it because you wouldn\'t prove an alibi for fear of compromising somebody in ext

[{}, {}, {}, {}, {}, {}, {}, {}, {}, {}]

In [11]:
prompt = """<s>[INST] <<SYS>>
{}
<</SYS>>

{} [/INST]"""

system_prompt = """You always communicate using a valid python JSON string. You will be given a trope and a story. Analyze carefully if the given trope exists or not in the story. The expected output from you should follow the structure:\n 
    {
        "trope_exists": {'yes' if the trope exists, 'no' if the trope does not exist},
        "evidence": {reason why the trope exists or does not exist in the context of the story}
    }
"""
user_msg = """\nTrope: {}\nStory:\n{}"""
def check_trope_exists(row):
    print(row)
    story = row[0]
    tropes = row[1]
    tropes_validated = []
    for trope in tropes:
        trope_name, trope_def = get_trope_by_id(trope)
        splitted = re.sub('([A-Z][a-z]+)', r' \1', re.sub('([A-Z]+)', r' \1', trope_name)).split()
        trope_name = " ".join(splitted)
        u_m = user_msg.format(trope_name.strip(), story.strip())
        # form the prompt
        p = prompt.format(system_prompt, u_m)
        # query the model
        # generate(p, temp=0.7)
        out = pipe_generate(p, temp=0.7)
        out = out[0]
        print(out)
        p_out = {}
        target = out.split('{')[1].split('}')[0]
        exists = target.split(',')[0].split(': ')[1].strip()
        p_out['tname'] = trope_name
        p_out['exists'] = strtobool(exists)
        p_out['reason'] = target.split(',',1)[1].split(':',1)[1].split('}')[0].strip()
        print(p_out)
        tropes_validated.append(p_out)
        # parse the output and save it to tropes_validated
    return tropes_validated
s = story_w_trope[story_w_trope['sid']=="s47"].iloc[0].story
# print(s)
t = story_w_trope[story_w_trope['sid']=="s47"].iloc[0].tropes
check_trope_exists((s,t))

('One time two clowns set themselves to arguing. The people\nwould stop, amused, to watch them.\n— What’s that? they asked.\n— Why, it’s only two clowns arguing.\nWho could take them seriously? Ridiculous, the two comedians rep-\narteed. The arguments were common nonsense, the theme was a ninnery.\nAnd an entire day passed.\nThe following morning, the two remained, obnoxious and outdoing\neach other. It seemed as though, between them, even yucca soured. In the\nstreet, meanwhile, those present were exhilarated with the masquerade.\nThe buffoons began worsening their insults with fine-edged and fine-\ntuned barbs. Believing it to be a show, the passersby left coins along the\nroadside.\nOn the third day, however, the clowns arrived at acts of force. Their\nblows became a disarray, their counterkicks zinged more across air than\nacross bodies. The children rollicked, imitating each jester’s blows. And\nthey laughed at the two fools, their bodies tripping upon their own selves.\nAnd the b

KeyboardInterrupt: 

In [83]:
prompt = """<s>[INST] <<SYS>>
{}
<</SYS>>

{} [/INST]"""

system_prompt = """You are an expert on tropes and you always answer truthfully. You always communicate using a valid python dictionary. You will be given a trope and a story and you analyze carefully if the given trope exists or not in the story. The expected output from you should follow the structure:\n 
    {
        "trope_exists": {'yes' if the trope exists, 'no' if the trope does not exist},
        "evidence": <reason critically why the trope exists or does not exist strictly in the context of the story>
    }
"""
user_msg = """\nTrope: {}\nStory:\n{}"""
def check_trope_exists(row):
    print(row)
    story = row[0]
    tropes = row[1]
    tropes_validated = []
    for trope in tropes:
        trope_name, trope_def = get_trope_by_id(trope)
        splitted = re.sub('([A-Z][a-z]+)', r' \1', re.sub('([A-Z]+)', r' \1', trope_name)).split()
        trope_name = " ".join(splitted)
        u_m = user_msg.format(trope_name.strip(), story.strip())
        # form the prompt
        p = prompt.format(system_prompt, u_m)
        # query the model
        # generate(p, temp=0.7)
        out = pipe_generate(p, temp=0.7)
        out = out[0]
        print(out)
        p_out = {}
        target = out.split('{')[1].split('}')[0]
        exists = target.split(',')[0].split(': ')[1].strip()
        p_out['tname'] = trope_name
        p_out['exists'] = strtobool(exists)
        p_out['reason'] = target.split(',',1)[1].split(':',1)[1].split('}')[0].strip()
        print(p_out)
        tropes_validated.append(p_out)
        # parse the output and save it to tropes_validated
    return tropes_validated
s = story_w_trope[story_w_trope['sid']=="s83"].iloc[0].story
# print(s)
t = story_w_trope[story_w_trope['sid']=="s83"].iloc[0].tropes
check_trope_exists((s,t))

('Nobody knew what to do at first. Deposit slips fluttered to the floor. A man touched the rim of his baseball cap to make sure it was still there.\n\nThe bank robber seemed just as shocked. There were too many people. He realized that, despite all his preparation, he hadn’t considered the afternoon rush.\n\n“Put your hands up,” he said, and people more or less did that. “You’re all hostages now.”\n\n“We don’t want to be hostages,” someone said.\n\n“That’s not how this works,” the bank robber said. “I come in, you all become hostages, I get some money and leave, and then you’re free again.”\n\n“We’re afraid,” someone else said.\n\nThe bank robber got it—he was also afraid.\n\nA bank teller waved her hand, which was still raised above her head. “I’ll be the hostage,” she said.\n\n“You are already one of many hostages,” the bank robber said.\n\n“Then everyone can go and sit in the old vault,” she said. “There’s no way to call for help in there, and it has a heavy iron door. That means fe

[{'tname': 'Big Bad',
  'exists': 1,
  'reason': "The character of the bank robber fits the description of the Big Bad trope. He is the main antagonist of the story, and his actions and behavior exhibit many of the characteristics associated with the trope. He is portrayed as incompetent, unsure of himself, and unprepared for the situation he finds himself in. He relies on the teller's assistance, who seems to know more about robbing banks than he does. He is also shown to be inconsiderate and ungrateful towards the teller, who is hostage and helping him. Moreover, the teller's comment about the bank robber's inability to understand the future or to confront it, suggests that he lacks the intelligence, courage, and determination that are typically associated with the Big Bad trope."},
 {'tname': 'Dramatic Irony',
  'exists': 1,
  'reason': "The story contains the trope of dramatic irony. The bank robber is unaware that the teller is secretly helping him, and the teller is aware of the 

In [87]:
prompt = """<s>[INST] <<SYS>>
{}
<</SYS>>

{} [/INST]"""

system_prompt = """You always answer truthfully and communicate using a valid JSON string. You will be given a trope and a story and you analyze carefully if the given trope exists or not in the story. First reason critically if the trope exists or not in the story. The expected output from you should follow the structure:\n
    {
        "trope_exists": {'yes' if the trope exists, 'no' if the trope does not exist},
        "evidence": {provide reason(s) strictly from the context of the story}
    }
"""
user_msg = """Trope: {}\nStory:\n{}"""
def check_trope_exists(row):
    print(row)
    story = row[0]
    tropes = row[1]
    tropes_validated = []
    for trope in tropes:
        trope_name, trope_def = get_trope_by_id(trope)
        splitted = re.sub('([A-Z][a-z]+)', r' \1', re.sub('([A-Z]+)', r' \1', trope_name)).split()
        trope_name = " ".join(splitted)
        u_m = user_msg.format(trope_name.strip(), story.strip())
        # form the prompt
        p = prompt.format(system_prompt, u_m)
        # query the model
        # generate(p, temp=0.7)
        out = pipe_generate(p, temp=0.55)
        out = out[0]
        print(out)
        p_out = {}
        # target = out.split('{')[1].split('}')[0]
        # exists = target.split(',')[0].split(': ')[1].strip()
        # p_out['tname'] = trope_name
        # p_out['exists'] = strtobool(exists)
        # p_out['reason'] = target.split(',',1)[1].split(':',1)[1].split('}')[0].strip()
        # print(p_out)
        tropes_validated.append(p_out)
        # parse the output and save it to tropes_validated
    return tropes_validated
s = story_w_trope[story_w_trope['sid']=="s83"].iloc[0].story
# print(s)
t = story_w_trope[story_w_trope['sid']=="s83"].iloc[0].tropes
check_trope_exists((s,t))

('Nobody knew what to do at first. Deposit slips fluttered to the floor. A man touched the rim of his baseball cap to make sure it was still there.\n\nThe bank robber seemed just as shocked. There were too many people. He realized that, despite all his preparation, he hadn’t considered the afternoon rush.\n\n“Put your hands up,” he said, and people more or less did that. “You’re all hostages now.”\n\n“We don’t want to be hostages,” someone said.\n\n“That’s not how this works,” the bank robber said. “I come in, you all become hostages, I get some money and leave, and then you’re free again.”\n\n“We’re afraid,” someone else said.\n\nThe bank robber got it—he was also afraid.\n\nA bank teller waved her hand, which was still raised above her head. “I’ll be the hostage,” she said.\n\n“You are already one of many hostages,” the bank robber said.\n\n“Then everyone can go and sit in the old vault,” she said. “There’s no way to call for help in there, and it has a heavy iron door. That means fe

[{}, {}, {}, {}, {}, {}, {}, {}, {}, {}]

In [12]:
import re

In [13]:
prompt = """<s>[INST] <<SYS>>
{}
<</SYS>>

{} [/INST]"""

system_prompt = """You always answer truthfully and communicate using a valid JSON string. You will be given a trope, its definition and a story and you analyze carefully if the given trope exists or not in the story. First reason critically if the trope exists or not in the story based on its definition. The expected output from you should follow the structure:\n
    {
        "trope_exists": {'yes' if the trope exists, 'no' if the trope does not exist},
        "evidence": {provide reason(s) strictly from the context of the story}
    }
"""
user_msg = """Trope: {}\nTrope Definition: {}\nStory:\n{}"""
def check_trope_exists(row):
    print(row)
    story = row[0]
    tropes = row[1]
    tropes_validated = []
    for trope in tropes:
        trope_name, trope_def = get_trope_by_id(trope)
        splitted = re.sub('([A-Z][a-z]+)', r' \1', re.sub('([A-Z]+)', r' \1', trope_name)).split()
        trope_name = " ".join(splitted)
        u_m = user_msg.format(trope_name.strip(), trope_def.strip(), story.strip())
        # form the prompt
        p = prompt.format(system_prompt, u_m)
        # query the model
        # generate(p, temp=0.7)
        out = pipe_generate(p, temp=0.55)
        out = out[0]
        print(trope_name, out)
        p_out = {}
        # target = out.split('{')[1].split('}')[0]
        # exists = target.split(',')[0].split(': ')[1].strip()
        # p_out['tname'] = trope_name
        # p_out['exists'] = strtobool(exists)
        # p_out['reason'] = target.split(',',1)[1].split(':',1)[1].split('}')[0].strip()
        # print(p_out)
        tropes_validated.append(p_out)
        # parse the output and save it to tropes_validated
    return tropes_validated
s = story_w_trope[story_w_trope['sid']=="s83"].iloc[0].story
# print(s)
t = story_w_trope[story_w_trope['sid']=="s83"].iloc[0].tropes
check_trope_exists((s,t))

('Nobody knew what to do at first. Deposit slips fluttered to the floor. A man touched the rim of his baseball cap to make sure it was still there.\n\nThe bank robber seemed just as shocked. There were too many people. He realized that, despite all his preparation, he hadn’t considered the afternoon rush.\n\n“Put your hands up,” he said, and people more or less did that. “You’re all hostages now.”\n\n“We don’t want to be hostages,” someone said.\n\n“That’s not how this works,” the bank robber said. “I come in, you all become hostages, I get some money and leave, and then you’re free again.”\n\n“We’re afraid,” someone else said.\n\nThe bank robber got it—he was also afraid.\n\nA bank teller waved her hand, which was still raised above her head. “I’ll be the hostage,” she said.\n\n“You are already one of many hostages,” the bank robber said.\n\n“Then everyone can go and sit in the old vault,” she said. “There’s no way to call for help in there, and it has a heavy iron door. That means fe

[{}, {}, {}, {}, {}, {}, {}, {}, {}, {}]

In [14]:
prompt = """<s>[INST] <<SYS>>
{}
<</SYS>>

{} [/INST]"""

system_prompt = """You always answer truthfully and communicate using a valid JSON string. You will be given a trope, its definition and a story and you analyze carefully if the given trope exists or not in the story. First reason critically if the trope exists or not in the story based on its definition. The expected output from you should follow the structure:\n
    {
        "trope_exists": {'yes' if the trope exists, 'no' if the trope does not exist},
        "evidence": {provide reason(s) strictly from the context of the story}
    }
"""
user_msg = """Trope: {}\nTrope Definition: {}\nStory:\n{}"""
def check_trope_exists(row):
    print(row)
    story = row[0]
    tropes = row[1]
    tropes_validated = []
    for trope in tropes:
        trope_name, trope_def = get_trope_by_id(trope)
        splitted = re.sub('([A-Z][a-z]+)', r' \1', re.sub('([A-Z]+)', r' \1', trope_name)).split()
        trope_name = " ".join(splitted)
        u_m = user_msg.format(trope_name.strip(), trope_def.strip(), story.strip())
        # form the prompt
        p = prompt.format(system_prompt, u_m)
        # query the model
        # generate(p, temp=0.7)
        out = pipe_generate(p, temp=0.7)
        out = out[0]
        print(out)
        p_out = {}
        # target = out.split('{')[1].split('}')[0]
        # exists = target.split(',')[0].split(': ')[1].strip()
        # p_out['tname'] = trope_name
        # p_out['exists'] = strtobool(exists)
        # p_out['reason'] = target.split(',',1)[1].split(':',1)[1].split('}')[0].strip()
        # print(p_out)
        tropes_validated.append(p_out)
        # parse the output and save it to tropes_validated
    return tropes_validated
s = story_w_trope[story_w_trope['sid']=="s83"].iloc[0].story
# print(s)
t = story_w_trope[story_w_trope['sid']=="s83"].iloc[0].tropes
check_trope_exists((s,t))

('Nobody knew what to do at first. Deposit slips fluttered to the floor. A man touched the rim of his baseball cap to make sure it was still there.\n\nThe bank robber seemed just as shocked. There were too many people. He realized that, despite all his preparation, he hadn’t considered the afternoon rush.\n\n“Put your hands up,” he said, and people more or less did that. “You’re all hostages now.”\n\n“We don’t want to be hostages,” someone said.\n\n“That’s not how this works,” the bank robber said. “I come in, you all become hostages, I get some money and leave, and then you’re free again.”\n\n“We’re afraid,” someone else said.\n\nThe bank robber got it—he was also afraid.\n\nA bank teller waved her hand, which was still raised above her head. “I’ll be the hostage,” she said.\n\n“You are already one of many hostages,” the bank robber said.\n\n“Then everyone can go and sit in the old vault,” she said. “There’s no way to call for help in there, and it has a heavy iron door. That means fe

/cluster/home/sharaj/.local/lib/python3.9/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


#####################################################
  {
"trope_exists": true,
"evidence": [
"The bank robber is the cause of all the problems in the story, as he is the one who initiates the robbery and takes the hostages."
"The teller, who is the main character, is the one who suggests the plan to rob the bank and helps the robber carry it out, making her the Big Bad of the story."
"The teller's motivations and backstory are not explored in detail, adding to the mystery and complexity of her character, which is a common trait of the Big Bad."
"The teller's actions and decisions drive the plot forward and create tension and conflict for the other characters, which is another hallmark of the Big Bad."
]
}
#####################################################
  {
"trope_exists": "yes",
"evidence": [
"The bank robber seems just as shocked as the hostages, indicating that he didn't consider the afternoon rush and is now improvising.",
"The teller suggests that the hostages file into the 

[{}, {}, {}, {}, {}, {}, {}, {}, {}, {}]

In [5]:
def strtobool (val):
    """Convert a string representation of truth to true (1) or false (0).
    True values are 'y', 'yes', 't', 'true', 'on', and '1'; false values
    are 'n', 'no', 'f', 'false', 'off', and '0'.  Raises ValueError if
    'val' is anything else.
    """
    val = val.lower()
    if val in ('y', 'yes', 't', 'true', 'on', '1', '"yes"', "'yes'"):
        return 1
    elif val in ('n', 'no', 'f', 'false', 'off', '0', '"no"', "'no'"):
        return 0
    else:
        raise ValueError("invalid truth value %r" % (val,))

In [38]:
# prompt = """<s>[INST] <<SYS>>
# {}
# <</SYS>>

# {} [/INST]"""

# system_prompt = """You always communicate using a valid JSON string. You will be given a trope, and a story and you must find out if the story makes use of the trope or not. The expected output from you should follow the structure:\n
#     {
#         "trope_exists": {'yes' if there is clear evidence of the given trope in the story, 'no' if the story does not use the trope},
#         "reason": {strictly in the context of the story, provide evidence if the trope exists, otherwise explain why the trope is not part of the story}
#     }
# """
# user_msg = """Trope: {}\nStory:\n{}"""
torch.cuda.empty_cache()
# Use a pipeline for later
pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer,
                torch_dtype=torch.float16,
                device_map="auto",
                do_sample=True,
                top_k=5,
                num_return_sequences=1,
                eos_token_id=tokenizer.eos_token_id,
                max_length=4096,
                return_full_text=False,
                # temperature=0.4 # ??
                )
# print(pipeline)
# print("######")
# @print_function_name
def generate(prompt, temp=0.6):
    with torch.autocast("cuda", dtype=torch.float16):
        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
        outputs = model.generate(
            **inputs,
            max_new_tokens=4096,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.eos_token_id,
            temperature = temp #??
        )
        final_outputs = tokenizer.batch_decode(outputs, skip_special_tokens=True)[0]
        #       generated_text = final_outputs[len(prompt):]
        # final_outputs = cut_off_text(final_outputs, '</s>')
        # print(final_outputs)
        torch.cuda.empty_cache() 
        print("#####################################################")
        return final_outputs

# @print_function_name
def pipe_generate(prompt, temp=0.6):
    sequences = pipe(prompt, temperature=temp)
    gen_text = []
    for seq in sequences:
        gen_text.append(seq['generated_text'])
        # print(seq['generated_text'])
    torch.cuda.empty_cache() 
    print("#####################################################")
    return gen_text
prompt = """<s>[INST] <<SYS>>
{}
<</SYS>>

{} [/INST]"""

system_prompt = """You always answer truthfully and communicate using a valid JSON string. You will be given a trope and a story and you analyze carefully if the given trope exists or not in the story. First reason critically if the trope exists or not in the story. The expected output from you should follow the structure:\n
    {
        "trope_exists": {'yes' if the trope exists in the story, 'no' if the trope does not exist in the story},
        "evidence": {provide evidence strictly from the context of the story}
    }
"""
user_msg = """Trope: {}\nStory:\n{}"""
def check_trope_exists(row):
    print(row)
    story = row[0]
    tropes = row[1]
    tropes_validated = []
    for trope in tropes:
        trope_name, trope_def = get_trope_by_id(trope)
        splitted = re.sub('([A-Z][a-z]+)', r' \1', re.sub('([A-Z]+)', r' \1', trope_name)).split()
        trope_name = " ".join(splitted)
        u_m = user_msg.format(trope_name.strip(), story.strip())
        # form the prompt
        p = prompt.format(system_prompt, u_m)
        # query the model
        # generate(p, temp=0.7)
        # print(p)
        out = pipe_generate(p, temp=0.45)# 0.50, .55 does a good job but it still hallucinates, 0.45 is great, 0.35 is also good #stick to 0.45
        out = out[0]
        print(trope_name, out)
        p_out = {}
        target = out.split('{')[1].split('}')[0]
        exists = target.split(',')[0].split(': ')[1].strip()
        p_out['tname'] = trope_name
        p_out['exists'] = strtobool(exists)
        p_out['reason'] = target.split(',',1)[1].split(':',1)[1].split('}')[0].strip()
        print(p_out)
        tropes_validated.append(p_out)
        # parse the output and save it to tropes_validated
    return tropes_validated
s = story_w_trope[story_w_trope['sid']=="s83"].iloc[0].story
# print(s)
t = story_w_trope[story_w_trope['sid']=="s83"].iloc[0].tropes
check_trope_exists((s,t))

('Nobody knew what to do at first. Deposit slips fluttered to the floor. A man touched the rim of his baseball cap to make sure it was still there.\n\nThe bank robber seemed just as shocked. There were too many people. He realized that, despite all his preparation, he hadn’t considered the afternoon rush.\n\n“Put your hands up,” he said, and people more or less did that. “You’re all hostages now.”\n\n“We don’t want to be hostages,” someone said.\n\n“That’s not how this works,” the bank robber said. “I come in, you all become hostages, I get some money and leave, and then you’re free again.”\n\n“We’re afraid,” someone else said.\n\nThe bank robber got it—he was also afraid.\n\nA bank teller waved her hand, which was still raised above her head. “I’ll be the hostage,” she said.\n\n“You are already one of many hostages,” the bank robber said.\n\n“Then everyone can go and sit in the old vault,” she said. “There’s no way to call for help in there, and it has a heavy iron door. That means fe

[{'tname': 'Big Bad',
  'exists': 0,
  'reason': '"The story does not contain any elements of the Big Bad trope. The character described as the bank robber is portrayed as incompetent, uncertain, and unconfident, lacking the typical characteristics of a Big Bad such as intelligence, cunning, and a clear motive. The teller, who is described as being knowledgeable and helpful, does not fit the role of a Big Bad either. The story does not have a clear antagonist or a sense of danger and urgency, which are also hallmarks of the Big Bad trope."'},
 {'tname': 'Dramatic Irony',
  'exists': 1,
  'reason': '[\n"The bank robber seemed just as shocked. There were too many people. He realized that, despite all his preparation, he hadn’t considered the afternoon rush.",\n"The teller turned an iron wheel to lock them in. \'All right,\' she said. \'Let’s get to work.\'",\n"He pulled back the leather chair behind one of the desks and sat. It felt funny to be behind a desk, dressed as he was in black f

## Same analysis as above but also pass the trope definition to see if it is able to correct its mistakes